In [4]:
#https://github.com/entbappy/Branching-tutorial/raw/master/Chicken-fecal-images.zip

import os
from pathlib import Path
from dataclasses import dataclass
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir:Path
    source_URL:str
    local_data_file:Path
    unzip_dir: Path 


In [13]:
%pwd

'c:\\Users\\jayak\\gitrepo\\learning\\chicken_disease_classification'

In [33]:
from chicken_disease_classifier.constants import *

from chicken_disease_classifier.utils.common import read_yaml,create_directories

class ConfigurationManager:
    def __init__(self,config_filepath=CONFIG_FILE_PATH,params_filepath=PARAMS_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self)->DataIngestionConfig:
        config=self.config.data_ingestion
        create_directories([config.root_dir])
        get_data_ingestion_config=DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return get_data_ingestion_config

In [22]:
#Components for loading data
import os
from pathlib import Path
import urllib.request as request
import zipfile
from chicken_disease_classifier import logger
from chicken_disease_classifier.utils.common import get_size

class DataIngestion:

    
    def __init__(self, config:DataIngestionConfig):
        self.config=config


    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename,headers=request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} download with following infor:\n{headers}")
        else:
            logger.info(f"File already exists pf size:{get_size(Path(self.config.local_data_file))}")


    def extract_zip_file(self):
        unzip_path=self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [35]:
#pipeline

config=ConfigurationManager()
data_ing_config=config.get_data_ingestion_config()
data_ing=DataIngestion(config=data_ing_config)
data_ing.download_file()
data_ing.extract_zip_file()


[2023-08-06 12:25:20,167:INFO:common:yaml file:config\config.yaml loaded successfully]
[2023-08-06 12:25:20,169:INFO:common:yaml file:params.yaml loaded successfully]
[2023-08-06 12:25:20,170:INFO:common:created directory at:artifacts]
[2023-08-06 12:25:20,171:INFO:common:created directory at:artifacts/data_ingestion]
[2023-08-06 12:25:21,594:INFO:2035662848:artifacts/data_ingestion/data.zip download with following infor:
Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: A2B4:6F25:12942F4:1360AD2:64CF8321
Accept-Ranges: bytes
Date: Sun, 06 Aug 2023 11:25:23 GMT
Via: 1.1 varnish
X-Served-By: cache-lhr7337-LHR
X-Cache: MISS
X-Cache-Hits: 0
